In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

Device: cpu


In [2]:
import numpy as np
import os
from get_data import get_data
from imblearn.over_sampling import SMOTE 
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.tensorflow import balanced_batch_generator
from numpy.random import default_rng
rng = default_rng(seed = 31)
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler 


In [3]:
from torch import nn, optim
import scipy.ndimage as ndimage

In [4]:
# X, Y, ID = get_data("../Data/filled/grids/", [2015,2016,2017,2018,2019])

In [5]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [6]:
with open('w15c.pickle', 'rb') as handle:
    wt = pickle.load(handle)
    
with open('w16c.pickle', 'rb') as handle:
    wv = pickle.load(handle)

In [7]:
def create_CNN_samples(grid, block, dims = 39):
    
    nonzero = np.transpose(grid[:,:,-2].nonzero()) # Get indices of nonzero componetns

    size = nonzero.shape[0]
    width = block * 2 + 1 # calculate widht and height. Needed later on
    
    X = np.zeros((size, width, width, dims))
    Y = np.zeros(size)
    ID = np.zeros(size)
    
    for idx, i in enumerate(nonzero):
        x, ID[idx], Y[idx] = get_neighbor_grid(grid, i, block)
        X[idx] = x.reshape(width,width, 39)
        
    X = np.moveaxis(X, -1, 1) # order the indices correctly to make sure it works in CNN
    X = torch.from_numpy(X).float()
    Y = torch.from_numpy(Y).float()
    
    return X,ID,Y

In [8]:
def get_neighbor_grid(full, hw, block = 1):
    
    # get the nonzero (built) blocks by checking if they have a ID

    h = hw[0]
    w = hw[1]
    
    y = full[h,w,-1]
    ID = full[h,w,-2]
    
    hu = h - block
    hd = h + block
    hshort, hextra, wshort, wextra = 0,0,0,0
    if hu < 0:
        hshort = 0 - hu
        hu = 0
    if hd >= full.shape[0]:
        hextra = (hd - full.shape[0]) + 1
        hd = full.shape[0]

    wr = w + block
    wl = w - block

    if wr >= full.shape[1]:
        wextra = (wr - full.shape[1]) + 1
        wr = full.shape[1]
    if wl < 0:
        wshort = 0 - wl
        wl = 0

    nb = full[hu : hd + 1, wl : wr + 1, :]
    nb = np.pad(nb, ((hshort, hextra), (wshort, wextra), (0,0)), mode = "constant", constant_values = 0)
    return nb[:,:,:-2], ID, y


In [9]:
X_train = []
Y_train = []
ID_train = []
for filename in os.listdir("../Data/filled/grids/2015/"):
    n = np.load("../Data/filled/grids/2015/" + filename)
    X, ID, Y = create_CNN_samples(n, 5)
    X_train.append(X)
    Y_train.append(Y)
    ID_train.append(ID)
    
Y_train = np.concatenate(Y_train)
ID_train = np.concatenate(ID_train)
X_train = np.concatenate(X_train)

X_train = np.moveaxis(X_train, 1, -1)
X_train = X_train.reshape(-1, 39)

X_train = ss.fit_transform(X_train)
X_train = X_train.reshape(-1, 11, 11, 39)

X_train = np.moveaxis(X_train, -1, 1)
X_train = torch.tensor(X_train).float()




X_val = []
Y_val = []
ID_val = []

for filename in os.listdir("../Data/filled/grids/2016/"):
    n = np.load("../Data/filled/grids/2016/" + filename)
    X, ID, Y = create_CNN_samples(n, 5)
    X_val.append(X)
    Y_val.append(Y)
    ID_val.append(ID)
    
X_val = np.concatenate(X_val)
X_val = np.moveaxis(X_val, 1, -1)
X_val = X_val.reshape(-1, 39)

X_val = ss.transform(X_val)


X_val = X_val.reshape(-1, 11, 11, 39)
X_val = np.moveaxis(X_val, -1, 1)
Y_val = np.concatenate(Y_val)
ID_val = np.concatenate(ID_val)
X_val = torch.tensor(X_val).float()

In [10]:
def get_batch(X, Y):
    
    y1 = np.argwhere(Y==1)
    percentage1 = len(y1) / len(X)
    while percentage1 < 0.4:
        X = np.append(X, X[y1].squeeze(), axis = 0)
        Y = np.append(Y, Y[y1].squeeze(), axis = 0) 
        percentage1 = (Y==1).sum() / len(X)
    

#     return X, Y


In [11]:
class Model2(nn.Module):
    def __init__(self, name):
        super(Model2, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 128, kernel_size = (3,3)), 
            nn.ReLU(), 
            nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = (3,3)), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(576, 64),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid())
            
        self.name = name
    def forward(self, x):
        out = self.net(x)
        return out

In [12]:
class Model3(nn.Module):
    def __init__(self, name):
        super(Model3, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 128, kernel_size = (3,3)), 
            nn.ReLU(), 
            nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = (3,3)), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(576, 128),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid())
            
        self.name = name
    def forward(self, x):
        out = self.net(x)
        return out

In [13]:
class Model4(nn.Module):
    def __init__(self, name):
        super(Model4, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 64, kernel_size = (3,3)), 
            nn.ReLU(), 
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3,3)), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1152, 512),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid())
            
        self.name = name
    def forward(self, x):
        out = self.net(x)
        return out

In [14]:
# model1 = Model1("een", w)
# model2 = Model2("twee")
# model3 = Model3("drie")
# model4 = Model4("vier")
# models = [model1, model2, model3, model4]

In [15]:
ID_train

array([2893058., 2870455., 2868833., ..., 1311910., 1176428., 1314791.])

In [16]:
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [17]:
class Model1(nn.Module):
    def __init__(self, name):
        super(Model1, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 32, kernel_size = (3,3)), 
            nn.ReLU(), 
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3,3)), 
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64, 1))
        
        self.final = nn.Sequential(
            nn.Sigmoid())
        
        
        self.name = name

       
    def forward(self, x, w, y, train = True):

        out = self.net(x)
        neighbors = [w.neighbors[x] for x in range(len(out))]
        transitions = [out[x] for x in neighbors]
        n_function = torch.zeros((len(out), w.max_neighbors + 1))
        for i, (t, idx) in enumerate(zip(transitions, range(len(out)))):
            n_function[i, 1:len(t) + 1] = t.squeeze()
            n_function[i, 0] = out[idx]      
        if train:
#             indices = np.arange(len(n_function)).reshape(-1,1)
#             under, y  = RandomUnderSampler().fit_resample(indices, y)
#             n_function = n_function[under].squeeze()
#             print(n_function)

            return n_function.mean(axis = 1), y.reshape(-1,1)
        
        
        else:
            return self.final(n_function.sum(axis =1 )), y
            
    
    def get_no_activation(self, x):
        out = self.net(x)
        return out

In [18]:
model = Model1("een")

In [19]:
def neighbor_part(model, X, idxs, w):
    preds = model(X)
    preds = pd.DataFrame(np.array(preds.detach())).set_index(idxs)
    
    neighbors = [w.neighbors[x] for x in idxs]
    transitions = [preds.loc[x].values for x in neighbors]



    n_function = np.zeros((len(preds), w.max_neighbors + 1))
    for i, (t, idx) in enumerate(zip(transitions, idxs)):
        n_function[i, 1:len(t) + 1] = t.squeeze()
        n_function[i, 0] = preds.loc[idx]
        
    return n_function

In [20]:
import time


# # optimizer = optim.RMSprop(model.parameters(), lr=0.001) 
# BCEloss = nn.BCELoss()
# model.train()

def train_model(model, X_train, Y_train, ID_train, wt, X_val, Y_val, ID_val, wv, num_epochs, batch_per_e = 1):
    optimizer = optim.RMSprop(model.parameters(), lr=0.01) 
    BCEloss = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(len(Y_train) / (Y_train == 1).sum()))
    train_loss = []
    train_loss_history = []
    acc_history = []
    ROC_history = []
    f1_score_history = []
    cmc_best = 0
    
    

    for epoch in range(num_epochs):
        epoch_loss = []
        print("epoch: {} of {}".format(epoch, num_epochs))
        
            
        model.train()

        optimizer.zero_grad()
        t0 = time.time()

        out, y_ = model(X_train, wt, Y_train)

        
        loss = BCEloss(out, torch.tensor(y_).float().squeeze())
        print(loss)
        loss.backward()
        optimizer.step()
        train_loss_history.append(loss)
                
        model.eval()
        
        out, y = model(X_val, wv, Y_val, False)

        out = out.detach().numpy()
        print(out)
        out =  (out > 0.5).astype(int)
        print(out)
        acc = accuracy_score(y, out)
        ROC = roc_auc_score(y, out)
        f1 = f1_score(y, out)

        acc_history.append(acc)
        ROC_history.append(ROC)
        train_loss_history.append(train_loss)
        f1_score_history.append(f1)
        
        
        print("training_loss: {:.4f}, acc: {:.3f}, ROC: {:.3f}, f1: {:.3f}".format(loss, acc, ROC , f1))
        train_loss = []


    return acc_history, ROC_history, train_loss_history, f1_score_history


        

In [21]:
out1 = train_model(model, X_train, Y_train, ID_train, wt, X_val, Y_val, ID_val, wv, 100)


epoch: 0 of 100
tensor(1.3268, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[1. 1. 1. ... 1. 1. 1.]
[1 1 1 ... 1 1 1]
training_loss: 1.3268, acc: 0.079, ROC: 0.500, f1: 0.147
epoch: 1 of 100
tensor(801.3655, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0]
training_loss: 801.3655, acc: 0.921, ROC: 0.500, f1: 0.000
epoch: 2 of 100
tensor(76.1194, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[1.6652913e-05 5.6664292e-13 4.1139433e-13 ... 4.3424500e-12 2.9497910e-06
 1.4462873e-06]
[0 0 0 ... 0 0 0]
training_loss: 76.1194, acc: 0.920, ROC: 0.500, f1: 0.001
epoch: 3 of 100
tensor(8.7281, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.8367634 1.        1.        ... 0.9999298 0.9101262 0.9746878]
[1 1 1 ... 1 1 1]
training_loss: 8.7281, acc: 0.079, ROC: 0.500, f1: 0.147
epoch: 4 of 100
tensor(5.6930, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[2.0806346e-09 1.7059128e-21 1.2888110e-21 ... 1.2567504e-18 4.7541404e-10
 4.3963999e-10]
[0 

[0.31408438 0.08500921 0.08178816 ... 0.0648412  0.23993276 0.23121771]
[0 0 0 ... 0 0 0]
training_loss: 1.2706, acc: 0.589, ROC: 0.577, f1: 0.179
epoch: 37 of 100
tensor(1.2702, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.40090904 0.24353571 0.23549342 ... 0.17504366 0.33753952 0.35421374]
[0 0 0 ... 0 0 0]
training_loss: 1.2702, acc: 0.428, ROC: 0.574, f1: 0.172
epoch: 38 of 100
tensor(1.2702, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.31163305 0.0836584  0.08050764 ... 0.0643174  0.23778409 0.2307973 ]
[0 0 0 ... 0 0 0]
training_loss: 1.2702, acc: 0.593, ROC: 0.578, f1: 0.179
epoch: 39 of 100
tensor(1.2694, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.40153304 0.24805033 0.23992659 ... 0.17975028 0.33918953 0.35706565]
[0 0 0 ... 0 0 0]
training_loss: 1.2694, acc: 0.425, ROC: 0.574, f1: 0.172
epoch: 40 of 100
tensor(1.2697, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.30855936 0.08125778 0.07821184 ... 0.06287358 0.23514363 0.22946432]
[0 0 0 ... 0 0 0]
tr

tensor(1.2728, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.4665425  0.53846514 0.5306407  ... 0.3508498  0.42033377 0.43913436]
[0 1 1 ... 0 0 0]
training_loss: 1.2728, acc: 0.308, ROC: 0.555, f1: 0.163
epoch: 74 of 100
tensor(1.2701, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.27958834 0.06487305 0.06336583 ... 0.04170964 0.2010312  0.18321398]
[0 0 0 ... 0 0 0]
training_loss: 1.2701, acc: 0.689, ROC: 0.581, f1: 0.187
epoch: 75 of 100
tensor(1.2693, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.45894286 0.51837474 0.51080334 ... 0.3340471  0.41159895 0.42918703]
[0 1 1 ... 0 0 0]
training_loss: 1.2693, acc: 0.330, ROC: 0.559, f1: 0.165
epoch: 76 of 100
tensor(1.2664, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.28552932 0.07219663 0.07065065 ... 0.04627083 0.20775239 0.1904513 ]
[0 0 0 ... 0 0 0]
training_loss: 1.2664, acc: 0.682, ROC: 0.581, f1: 0.187
epoch: 77 of 100
tensor(1.2667, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.45402044 0.50587624 0.4985

In [28]:
out1 = train_model(model, X_train, Y_train, ID_train, wt, X_val, Y_val, ID_val, wv, 1000)

epoch: 0 of 1000
tensor(1.2410, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[1. 1. 1. ... 1. 1. 1.]
[1 1 1 ... 1 1 1]
training_loss: 1.2410, acc: 0.079, ROC: 0.500, f1: 0.147
epoch: 1 of 1000
tensor(113.6155, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.05392889 0.00064442 0.00063491 ... 0.00177575 0.04499425 0.04129361]
[0 0 0 ... 0 0 0]
training_loss: 113.6155, acc: 0.910, ROC: 0.515, f1: 0.074
epoch: 2 of 1000
tensor(1.5731, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.995981  1.        1.        ... 0.999997  0.9977756 0.9984238]
[1 1 1 ... 1 1 1]
training_loss: 1.5731, acc: 0.079, ROC: 0.500, f1: 0.147
epoch: 3 of 1000
tensor(3.5810, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.4730444  0.39475468 0.39587796 ... 0.44473293 0.47315583 0.4730444 ]
[0 0 0 ... 0 0 0]
training_loss: 3.5810, acc: 0.557, ROC: 0.583, f1: 0.180
epoch: 4 of 1000
tensor(1.2719, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
[0.47307694 0.38558444 0.3866775  ... 0.43232006 0.47307694 0.

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1091265120 bytes. Buy new RAM!
00007FFF7D8A783900007FFF7D8A77F0 c10.dll!c10::ThrowEnforceNotMet [<unknown file> @ <unknown line number>]
00007FFF7D88A56700007FFF7D88A3F0 c10.dll!c10::alloc_cpu [<unknown file> @ <unknown line number>]
00007FFF7D88A6E200007FFF7D88A6C0 c10.dll!c10::DefaultCPUAllocator::allocate [<unknown file> @ <unknown line number>]
00007FFF7D886B5000007FFF7D886B20 c10.dll!c10::Allocator::raw_allocate [<unknown file> @ <unknown line number>]
00007FFF1B7E734B00007FFF1B7E6720 torch_cpu.dll!at::native::mkldnn_convolution_backward_weights [<unknown file> @ <unknown line number>]
00007FFF1B7DB30B00007FFF1B7D4A10 torch_cpu.dll!at::native::_fft_mkl [<unknown file> @ <unknown line number>]
00007FFF1B7DC8CE00007FFF1B7DBC20 torch_cpu.dll!at::native::mkldnn_mul_out [<unknown file> @ <unknown line number>]
00007FFF1B7DDE6000007FFF1B7DBC20 torch_cpu.dll

In [ ]:
out1

In [ ]:
l = nn.BCELoss()

In [ ]:
torch.tensor(Y_train)

In [ ]:
l(out, torch.tensor(Y_train))

In [ ]:
out1.shape

In [ ]:

hists = train_model(models[0], X_train, Y_train, ID_train, wt, X_val, Y_val, ID_val, 1)
for mod in models:
    hists = train_model(mod, X_train, Y_train, ID_train, wt, X_val, Y_val, ID_val, 1)
    n_function = neighbor_part(mod, X_train, ID_train, wt)
        
    oversample = SMOTE()
    x, y = oversample.fit_resample(n_function, Y_train)

    clf_bagger = RandomForestClassifier(max_depth = 12, oob_score = True)
    clf_bagger.fit(x,y)
    
    n_function = neighbor_part(mod, X_val, ID_val, wv)
    preds = clf_bagger.predict(n_function)
    totacc = accuracy_score(Y_val, preds)
    totf1 = f1_score(Y_val, preds)
    totROC = roc_auc_score(Y_val, preds)
    
    
#     with open("../results/CNN/" + mod.name + ".csv", "a+") as f:
#         f.write("loss;acc;ROC;f1_score\n")
#         f.write(str(hists[2]) + ";" + str(hists[0]) + ";" + str(hists[1]) + ";" + str(hists[3]) + "\n")
#         f.write("--;" + str(totacc) + ";" + str(totROC) + ";" + str(totf1))
    
    
    